In [1]:
import pandas as pd
import numpy as np


In [2]:
!pip install pyarrow
import pyarrow as pa

In [3]:
from pyarrow import json as pajson

pages = pajson.read_json("dense-articles.json.gz").to_pandas()
pages = pages.astype({'id':'int32', 'lat':'float32', 'lng':'float32'}).set_index('id').sort_index()

In [4]:
revisions = pajson.read_json("dense-revisions.json.gz").to_pandas()
revisions = revisions.groupby(['page','user']).size().rename('count').reset_index()
revisions = revisions.astype({'page': 'int32', 'user': 'int32', 'count': 'int32'})

In [5]:
with open('pages.arrow','wb') as f:
    f.write(pa.default_serialization_context().serialize(pages).to_buffer())

In [6]:
with open('revisions.arrow','wb') as f:
    f.write(pa.default_serialization_context().serialize(revisions).to_buffer())

In [7]:
pages

,title,lat,lng
id,,,
1,United States,40.000000,-100.000000
2,India,21.000000,78.000000
3,United Kingdom,55.000000,-3.000000
4,New York City,40.660999,-73.944000
5,Canada,60.000000,-95.000000
...,...,...,...
1169056,Schachen (Gais) railway station,47.361526,9.477222
1169057,Hebrig railway station,47.361946,9.465000
1169058,Gais railway station,47.361111,9.451111


In [8]:
data = [pa.array(pages[c]) for c in pages.columns]
batch = pa.RecordBatch.from_arrays(data, pages.columns)
sink = pa.BufferOutputStream()
writer = pa.RecordBatchStreamWriter(sink, batch.schema)
writer.write_batch(batch)
writer.close()
with open('pages.noindex.arrow', 'wb') as f:
    f.write(sink.getvalue())